In [1]:
%load_ext autoreload
%autoreload 2

from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.qdc_experiment import QDCExperiment
from qdc.misc import tnow
import os 
DATA_DIR = os.path.join(os.path.abspath(os.path.curdir), 'data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Construct the multi-wavelength fiber set
fiber_L = 0.25e6  # um 
N_wl = 81
N_classical = 5
N_klyshkos = 5
wl0 = 0.810
Dwl = 0.080
# dzs = [0, 20, 80]
dzs = [0]

s = ManyWavelengthFiber(wl0=wl0, Dwl=Dwl, N_wl=N_wl, fiber_L=fiber_L, rng_seed=5555)

# Construct the experiment with that ManyWavelengthFiber
exp = QDCExperiment(s)

# Now run a multi-dz test, returning a QDCResult
res = exp.run_PCCs_different_dz(dzs=dzs, N_classical=N_classical, N_klyshko=N_klyshkos)

# Plot the result
res.show(title="")

saveto_path = rf'{DATA_DIR}\\{tnow()}.npz'
# Optionally save the result to disk
res.saveto(saveto_path)

Getting 81 fibers...


81it [00:07, 10.78it/s]


Got fibers!
Getting classical with average on 5 ...


100%|██████████| 5/5 [00:36<00:00,  7.21s/it]


Getting Klyshko with average on 5, dz=0 ...


100%|██████████| 5/5 [00:36<00:00,  7.22s/it]


In [20]:
res.show(title='', saveto_path=r'C:\temp\MMF_sim')